In [2]:
import numpy as np
import pandas as pd
import os
#import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras import models
from keras import layers

from keras import Sequential
from keras.layers import Dense, Activation



Using TensorFlow backend.


In [3]:
def read_feather():
    train_df = pd.read_feather('/Users/kimnan-young/data/train.ftr')
    test_df = pd.read_feather('/Users/kimnan-young/data/test.ftr')
    
    return train_df, test_df


In [5]:
train, test = read_feather()


In [6]:
train.head()

,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,type,long_GMI,lat_GMI,long_DPR,lat_DPR,precipitation
0,168.745071,90.459198,193.079697,125.343018,226.008530,215.537094,152.781219,267.132385,237.999283,0,160.967621,40.126064,161.071915,40.070358,0.0
1,166.973694,89.801224,193.049561,125.630112,225.944229,215.421158,152.503128,266.770691,238.626785,0,161.015671,40.088432,161.071915,40.070358,0.0
2,166.841629,90.226326,192.859787,125.389030,225.677124,216.231277,152.096664,267.000214,238.367294,0,161.064255,40.051228,161.071915,40.070358,0.0
3,167.142822,89.574203,193.425949,125.622368,226.249557,215.538849,152.664124,266.966583,237.810135,0,161.113358,40.014450,161.097397,40.003548,0.0
4,167.638077,90.122505,192.978592,125.161728,226.883911,216.013626,152.471634,267.064331,237.794601,0,161.162994,39.978111,161.151382,39.975197,0.0


In [7]:
x_train = train.drop(['precipitation'], axis = 1)
y_train = train['precipitation']

In [8]:
x_train.head()

,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,type,long_GMI,lat_GMI,long_DPR,lat_DPR
0,168.745071,90.459198,193.079697,125.343018,226.008530,215.537094,152.781219,267.132385,237.999283,0,160.967621,40.126064,161.071915,40.070358
1,166.973694,89.801224,193.049561,125.630112,225.944229,215.421158,152.503128,266.770691,238.626785,0,161.015671,40.088432,161.071915,40.070358
2,166.841629,90.226326,192.859787,125.389030,225.677124,216.231277,152.096664,267.000214,238.367294,0,161.064255,40.051228,161.071915,40.070358
3,167.142822,89.574203,193.425949,125.622368,226.249557,215.538849,152.664124,266.966583,237.810135,0,161.113358,40.014450,161.097397,40.003548
4,167.638077,90.122505,192.978592,125.161728,226.883911,216.013626,152.471634,267.064331,237.794601,0,161.162994,39.978111,161.151382,39.975197


In [9]:
y_train

0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
            ... 
122151995    0.0
122151996    0.0
122151997    0.0
122151998    0.0
122151999    0.0
Name: precipitation, Length: 122152000, dtype: float32

In [10]:
test.head()

,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,type,long_GMI,lat_GMI,long_DPR,lat_DPR
0,166.479080,89.160690,187.936172,118.861412,214.203506,214.023972,151.760315,263.783478,233.330048,0,131.104523,40.599606,131.103607,40.573402
1,166.881927,89.890549,189.061615,119.168121,214.135696,212.979538,150.284988,262.505219,231.299652,0,131.169418,40.618748,131.155701,40.600952
2,167.336197,89.116234,187.739044,119.040283,214.371033,213.933899,151.608582,263.448883,234.285645,0,131.234039,40.638447,131.207962,40.628593
3,166.961426,90.114693,188.004745,119.050346,213.354935,213.654694,152.032822,262.841278,232.018250,0,131.298370,40.658699,131.311569,40.683399
4,166.883255,89.838066,188.203415,119.100990,213.265976,213.429672,152.934494,263.799194,234.420868,0,131.362427,40.679501,131.362656,40.710442


## 평가 함수 정의

### 규칙에 정의된 함수



In [11]:
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

## 손실 함수 정의

### 베이스라인에 정의된 함수

In [12]:
def mae(y_true, y_pred):    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def score(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

## 모델 설계 및 학습
\


In [13]:
x_train.shape

(122152000, 14)

In [14]:
x_train = x_train.dropna()

In [ ]:
x_train.shpae

In [12]:
# model = Sequential()

# model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (122152000, 14)))
# model.add(layers.MaxPooling2D(2,2))
# model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
# model.add(layers.MaxPooling2D(2,2))
# model.add(layers.Conv2D(64,(3,3), activation = 'relu'))



# def build_model(): #동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다.
#     model = models.Sequential()
#     model.add(layers.Dense(64, activation='relu',
#                            input_shape=(122152000, 14)))
#     model.add(layers.Dense(64, activation='relu'))
#     model.add(layers.Dense(1))
#     model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
#     return model



In [ ]:
model.compile